# Multiple Linear Regression
We begin by loading the required libraries.

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sys

Considering subset of the entire dataset focused on people living in California above the age of 65: 

In [5]:
df=pd.read_csv('Data/elderly-ppl-CA.csv')
df.head()

,Unnamed: 0,age,gender,state,income,education,high_speed_internet_users,technology_and_connectivity__online_gamers,technology_and_connectivity__stream_music,technology_and_connectivity__science_&_new_tech_enthusiasts,...,chlamydia,gential_warts,gonorrhea,herpes,hpv,other_std,parasitic,std_screen,syphilis,trich
0,6010,65-74 years old,Male,California,"$190,000 - $199,999",High School,medium,0.057847,0.048718,0.171325,...,NaN,NaN,NaN,NaN,0.428571,NaN,NaN,1.000000,NaN,NaN
1,6011,65-74 years old,Male,California,"$190,000 - $199,999",High School,medium,0.057847,0.048718,0.171325,...,NaN,NaN,NaN,0.500000,0.500000,0.500000,NaN,1.166667,NaN,NaN
2,6012,65-74 years old,Male,California,"$190,000 - $199,999",High School,medium,0.057847,0.048718,0.171325,...,NaN,NaN,NaN,NaN,0.333333,0.333333,NaN,0.666667,NaN,0.333333
3,6013,65-74 years old,Male,California,"$190,000 - $199,999",High School,medium,0.057847,0.048718,0.171325,...,NaN,NaN,NaN,0.363636,0.272727,NaN,NaN,0.727273,NaN,NaN
4,6014,65-74 years old,Male,California,"$190,000 - $199,999",High School,medium,0.057847,0.048718,0.171325,...,NaN,0.333333,NaN,0.333333,NaN,0.444444,NaN,1.222222,0.333333,0.333333


Using just the columns with categorical variables: 

In [6]:
obj_df=df.select_dtypes(include=['object']).copy()
obj_df=obj_df.drop('date',axis=1)
string_cols=obj_df.columns
string_cols=list(string_cols)
print(string_cols)

['age', 'gender', 'state', 'income', 'education', 'high_speed_internet_users', 'wearable_technology_and_connectivity_users', 'smart_phone_users', 'restaurant_app_users', 'services_software_and_online_services_frequency', 'services_software_and_online_services_spend', 'restaurants_sit_down_casual_dining_frequency', 'restaurants_sit_down_casual_dining_spend', 'restaurants_sit_down_upscale_dining_frequency', 'restaurants_sit_down_upscale_dining_spend', 'restaurants_fast_casual_dining_frequency', 'restaurants_fast_casual_dining_spend', 'entertainment_movies_frequency', 'entertainment_movies_spend', 'services_ride_sharing_spend', 'services_ride_sharing_frequency', 'coffee_enthusiasts']


Performing one-hot encoding for these predictors: 

In [12]:
#one-hot encoding for these
final_df=pd.concat([df,pd.get_dummies(df[string_cols])],axis=1)

#convert dates to datetimes
final_df['date']=final_df['date'].astype('datetime64[ns]')

# print statement used for debugging 
#print(list(final_df.columns))
#print(list(final_df.dtypes))

We can now perform a multiple linear regression on each std:

In [8]:
# first get lists of dependent and independent variables

#dependent vars
std_list=['chlamydia','gential_warts','gonorrhea','herpes','hpv',
          'other_std','parasitic','syphilis','trich']

#independent vars
age_vars=['age_65-74 years old','age_75+ years old']
gender_vars=['gender_Female','gender_Male']
income_vars=list(final_df['income'].unique())
for i in range(len(income_vars)):
    temp=income_vars[i]
    temp2='income_'+temp
    income_vars[i]=temp2
education_vars=list(final_df['education'].unique())
for i in range(len(education_vars)):
    temp=education_vars[i]
    temp2='education_'+temp
    education_vars[i]=temp2

In [9]:
total_list=list(income_vars+education_vars+gender_vars)
arr=np.array(np.zeros((len(std_list),len(total_list))))

for i in range(len(std_list)):
    std=std_list[i]
    y=final_df[std]
    X=final_df[gender_vars+income_vars+education_vars]
    est=sm.OLS(y,X,missing='drop').fit()
    arr[i,:]=est.pvalues
    #print(est.summary()
for i in range(len(std_list)):
    std=std_list[i]
    for j in range(len(total_list)):
        var=total_list[j]
        print('\t'+ std + '\t'+var+ '\t'+ str(arr[i,j]))
alpha=0.05
bonferroni_p=alpha/len(list(income_vars+education_vars+gender_vars))
print(bonferroni_p)

	chlamydia	income_$190,000 - $199,999	2.1387551023983e-310
	chlamydia	income_$40,000 - $44,999	0.0
	chlamydia	income_$35,000 - $39,999	0.10876661412952421
	chlamydia	income_$170,000 - $179,999	0.002857892008171813
	chlamydia	income_$95,000 - $99,999	0.00013548369705892966
	chlamydia	income_$45,000 - $49,999	0.0015699311522930778
	chlamydia	income_$135,000 - $139,999	2.2284675438336647e-06
	chlamydia	income_$50,000 - $54,999	2.954461207793147e-35
	chlamydia	income_$105,000 - $109,999	0.12357981264686511
	chlamydia	income_$65,000 - $69,999	2.644902394319093e-22
	chlamydia	income_Less than $14,999	2.4797416302333314e-10
	chlamydia	income_$55,000 - $59,999	0.002566982973741658
	chlamydia	income_$85,000 - $89,999	3.2339803734722264e-27
	chlamydia	income_$140,000 - $144,999	0.0065792624243179824
	chlamydia	income_$150,000 - $159,999	0.4740148091052664
	chlamydia	income_$225,000 - $249,999	0.45103195231351334
	chlamydia	income_$90,000 - $94,999	1.2539925743558806e-16
	chlamydia	income_$180,00